# Coursera_Capstone project

This is the Coursera_Capstone project, the last one from the BM data science certification from Coursera. 

# 1. First Week - Creating Rep and simple Notebook

In [3]:
import pandas as pd
import numpy as np

In [4]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# 2. Third Week - Segmenting and Clustering Neighborhoods in Toronto

In [6]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [7]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.
